#Credit Card Fraud Detection
###by Daniel Pham

##README

When running this .ipynb file on google collab, you **MAY** have to restart the runtime in order to use the imblearn library.

If this is not being ran in any other IDE, make sure the imblearn library is installed in order for this code to be properly runned.

In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score

!pip install -U imbalanced-learn
from imblearn.over_sampling import SMOTE

Requirement already up-to-date: imbalanced-learn in /usr/local/lib/python3.7/dist-packages (0.8.0)


The credit card fraud problem has given us known labels so I will look to do a supervised technique.

Since the output is binary in nature (fraud or not fraud), I decided that I will use **logistic regression** as my method for this project.

We then can take a look at the weights to see what are the main features that contribute to fraud.

#Exploring the problem

In [ ]:
#if uploading the CSV to google collab, it takes a while for it to be fully uploaded so please wait
df = pd.read_csv("creditcard.csv")

In [ ]:
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [ ]:
df.describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,284807.000000,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,284807.000000,284807.000000
mean,94813.859575,3.919560e-15,5.688174e-16,-8.769071e-15,2.782312e-15,-1.552563e-15,2.010663e-15,-1.694249e-15,-1.927028e-16,-3.137024e-15,1.768627e-15,9.170318e-16,-1.810658e-15,1.693438e-15,1.479045e-15,3.482336e-15,1.392007e-15,-7.528491e-16,4.328772e-16,9.049732e-16,5.085503e-16,1.537294e-16,7.959909e-16,5.367590e-16,4.458112e-15,1.453003e-15,1.699104e-15,-3.660161e-16,-1.206049e-16,88.349619,0.001727
std,47488.145955,1.958696e+00,1.651309e+00,1.516255e+00,1.415869e+00,1.380247e+00,1.332271e+00,1.237094e+00,1.194353e+00,1.098632e+00,1.088850e+00,1.020713e+00,9.992014e-01,9.952742e-01,9.585956e-01,9.153160e-01,8.762529e-01,8.493371e-01,8.381762e-01,8.140405e-01,7.709250e-01,7.345240e-01,7.257016e-01,6.244603e-01,6.056471e-01,5.212781e-01,4.822270e-01,4.036325e-01,3.300833e-01,250.120109,0.041527
min,0.000000,-5.640751e+01,-7.271573e+01,-4.832559e+01,-5.683171e+00,-1.137433e+02,-2.616051e+01,-4.355724e+01,-7.321672e+01,-1.343407e+01,-2.458826e+01,-4.797473e+00,-1.868371e+01,-5.791881e+00,-1.921433e+01,-4.498945e+00,-1.412985e+01,-2.516280e+01,-9.498746e+00,-7.213527e+00,-5.449772e+01,-3.483038e+01,-1.093314e+01,-4.480774e+01,-2.836627e+00,-1.029540e+01,-2.604551e+00,-2.256568e+01,-1.543008e+01,0.000000,0.000000
25%,54201.500000,-9.203734e-01,-5.985499e-01,-8.903648e-01,-8.486401e-01,-6.915971e-01,-7.682956e-01,-5.540759e-01,-2.086297e-01,-6.430976e-01,-5.354257e-01,-7.624942e-01,-4.055715e-01,-6.485393e-01,-4.255740e-01,-5.828843e-01,-4.680368e-01,-4.837483e-01,-4.988498e-01,-4.562989e-01,-2.117214e-01,-2.283949e-01,-5.423504e-01,-1.618463e-01,-3.545861e-01,-3.171451e-01,-3.269839e-01,-7.083953e-02,-5.295979e-02,5.600000,0.000000
50%,84692.000000,1.810880e-02,6.548556e-02,1.798463e-01,-1.984653e-02,-5.433583e-02,-2.741871e-01,4.010308e-02,2.235804e-02,-5.142873e-02,-9.291738e-02,-3.275735e-02,1.400326e-01,-1.356806e-02,5.060132e-02,4.807155e-02,6.641332e-02,-6.567575e-02,-3.636312e-03,3.734823e-03,-6.248109e-02,-2.945017e-02,6.781943e-03,-1.119293e-02,4.097606e-02,1.659350e-02,-5.213911e-02,1.342146e-03,1.124383e-02,22.000000,0.000000
75%,139320.500000,1.315642e+00,8.037239e-01,1.027196e+00,7.433413e-01,6.119264e-01,3.985649e-01,5.704361e-01,3.273459e-01,5.971390e-01,4.539234e-01,7.395934e-01,6.182380e-01,6.625050e-01,4.931498e-01,6.488208e-01,5.232963e-01,3.996750e-01,5.008067e-01,4.589494e-01,1.330408e-01,1.863772e-01,5.285536e-01,1.476421e-01,4.395266e-01,3.507156e-01,2.409522e-01,9.104512e-02,7.827995e-02,77.165000,0.000000
max,172792.000000,2.454930e+00,2.205773e+01,9.382558e+00,1.687534e+01,3.480167e+01,7.330163e+01,1.205895e+02,2.000721e+01,1.559499e+01,2.374514e+01,1.201891e+01,7.848392e+00,7.126883e+00,1.052677e+01,8.877742e+00,1.731511e+01,9.253526e+00,5.041069e+00,5.591971e+00,3.942090e+01,2.720284e+01,1.050309e+01,2.252841e+01,4.584549e+00,7.519589e+00,3.517346e+00,3.161220e+01,3.384781e+01,25691.160000,1.000000


Since this problem is about credit card fraud, there is usually a large class imbalance, so let's take a look at how poor is the class imbalance.

In [ ]:
class_count = pd.value_counts(df['Class'])
print(class_count)

0    284315
1       492
Name: Class, dtype: int64


As shown, there is a huge class imbalance. Only 492 cases of fraud out of over 280,000 samples!

The algorithm may not be trained properly with too few samples of the minority class.

To solve this class imbalance, I will  oversample the miniority class (fraud) by **SMOTE** instead of undersampling the majority class (not fraud).

Furthermore, this data is already cleaned up for us by the providers. All the features except for a couple are converted into PCA values. The features that were not converted into PCA values may need to normalized but we'll see.

#Creating Training and Test Sets

To properly train a logistic regression model, we will first need to make 2 separate sets from the data: a training set to train the model and a test set to test how accurate the good the trained model is.

The training set will contain 80% of the data while the test set will contain the rest (20% of the data).

In [ ]:
training_set, test_set = train_test_split(df, test_size = 0.2)

#checking if it's done correctly

training_set.info()
test_set.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 227845 entries, 228044 to 158499
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    227845 non-null  float64
 1   V1      227845 non-null  float64
 2   V2      227845 non-null  float64
 3   V3      227845 non-null  float64
 4   V4      227845 non-null  float64
 5   V5      227845 non-null  float64
 6   V6      227845 non-null  float64
 7   V7      227845 non-null  float64
 8   V8      227845 non-null  float64
 9   V9      227845 non-null  float64
 10  V10     227845 non-null  float64
 11  V11     227845 non-null  float64
 12  V12     227845 non-null  float64
 13  V13     227845 non-null  float64
 14  V14     227845 non-null  float64
 15  V15     227845 non-null  float64
 16  V16     227845 non-null  float64
 17  V17     227845 non-null  float64
 18  V18     227845 non-null  float64
 19  V19     227845 non-null  float64
 20  V20     227845 non-null  float64
 21  V21  

Creating a variable that contains just *labels* for the samples for both training and test set to use for scikit-learn's library.

In [ ]:
training_labels = training_set['Class']
test_labels = test_set['Class']

#checking if it's done correctly
training_labels.head()

228044    0
13108     0
258199    0
83554     0
46084     0
Name: Class, dtype: int64

Dropping the 'Class' column to create a variable of just *features* for both training and test set.

In [ ]:
training_features = training_set.drop(['Class'], axis = 1)
test_features = test_set.drop(['Class'], axis = 1)

#checking if it's done correctly
training_features.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
228044,145335.0,2.065967,-0.327213,-2.221817,-0.713805,0.435340,-1.028121,0.451890,-0.285918,0.407632,0.016286,0.858679,0.489616,-1.233011,0.975564,-0.612040,-0.618106,-0.110769,-0.281928,0.741601,-0.197287,0.142837,0.469987,0.000287,0.792088,0.240474,1.027339,-0.156490,-0.093711,27.00
13108,23003.0,-0.250748,0.228252,1.997959,-0.668625,-0.653781,-0.404783,-0.247943,0.087898,2.572726,-1.660099,0.791595,-2.089710,1.127353,1.387759,0.860167,-1.212940,1.226730,0.228861,0.645060,-0.071120,0.050256,0.625550,0.084838,0.355037,-0.857608,-0.769203,0.269768,0.205272,11.85
258199,158539.0,-1.121313,0.771984,1.068908,-1.380343,0.517473,0.200858,0.500902,0.033177,1.298020,0.070078,-1.622049,-0.870944,-1.109992,-0.394997,0.651399,0.288610,-0.863619,0.131083,0.090543,0.198547,-0.380745,-0.619611,0.001221,-0.042411,-0.106341,-0.264659,0.317570,0.066396,7.68
83554,59914.0,-1.029989,1.269654,2.026286,0.984254,0.853755,0.189979,1.372185,-0.798930,-0.061175,2.021282,1.551693,-0.458205,-1.458916,-0.545649,-0.371239,0.431947,-1.267890,-0.298401,-1.891030,-0.012271,-0.042010,0.161717,-0.169810,0.141905,-0.391466,-0.469039,-1.348529,-0.787572,8.13
46084,42633.0,-0.335568,0.982971,1.595383,1.034390,0.292466,0.067714,1.041377,-0.185117,-1.021620,0.269348,1.988393,0.981614,0.405207,0.301178,0.742268,-0.538149,-0.102792,-0.535909,0.784026,0.109466,-0.333262,-0.862850,0.147012,0.156859,-0.410384,-0.765186,-0.207621,-0.193970,52.00


We have 4 new variables. Now we can refer to the features and labels of the training and test sets.

#Applying Scikit-learn's Logistic Regression **without** SMOTE sampling

First, we want to see how the logistic regression algorithm will fare without applying oversampling to the dataset.

In [ ]:
#create the trained algorithm by using the training set
logreg = LogisticRegression(max_iter=1000)
logreg.fit(training_features, training_labels)

#test the algorithm with the test set
logreg_score = logreg.score(test_features, test_labels)
print("Logistic regression accuracy without oversampling:", logreg_score)

Logistic regression accuracy without oversampling: 0.9991748885221726


Scikit-learn's library fitted the model to the dataset really well.

Now I will cross-validate the model over 10-folds of sampling to see if it returns similar accuracies.

In [ ]:
cv_scores = cross_val_score(logreg, training_features, training_labels, cv=10)

print("Cross Validation Scores without oversampling: \n", cv_scores)

cv_scores = pd.Series(cv_scores)
print("Cross Validation Minimum:", cv_scores.min())
print("Cross Validation Mean:", cv_scores.mean()) 
print("Cross Validation Max:", cv_scores.max())

Cross Validation Scores without oversampling: 
 [0.99938556 0.99921001 0.99868334 0.99916612 0.9992539  0.99916608
 0.99916608 0.9990783  0.99920997 0.99920997]
Cross Validation Minimum: 0.9986833443054641
Cross Validation Mean: 0.9991529332034554
Cross Validation Max: 0.9993855606758832


Looking at the cross validation scores, our model overall is having similar accuracy scores as to our trained accuracy. But is it actually predicting the class that we are interested in (the samples that are fraud) correctly?



It is time to look at the confusion matrix and F-scores

In [ ]:
#having the trained model predict over the test set
predictions = logreg.predict(test_features)

#confusion matrix from comparing the predictions to the data's actual labels
cm =  confusion_matrix(test_labels, predictions)

print("Confusion matrix without oversampling: \n", cm)

Confusion matrix without oversampling: 
 [[56839    18]
 [   29    76]]


To interpret this...

**Upper Left Corner**: The amount of correctly classified by the model for non-fraudulent transactions.

**Upper Right Corner**: The amount of incorrectly classified as fraudulent transactions, but the actual label is non-fraudulent.

**Lower Left Corner**: The amount of incorrectly classified as non-fraudulent transactions, but the actual label is fraudulent.

**Lower Right Corner**: The amount of correctly classified by the model for fraudulent transactions.

From an eyeball glance of the confusion matrix, the model is predicting very well for the non-fraudulent transactions. But that is not what we're interested looking at.

When taking a look at fraudulent cases for the confusion matrix, we are getting a rather unsatisfactory result as expected. The model is correctly predicting fraudulent cases at least 70% of the time. From an industry point of view, this model is most likely not good enough.

To further reinforce the results, let's take a look at the classification report for the cunfusion matrix.

In [ ]:
report = classification_report(test_labels, predictions)

print("Classification report without oversampling: \n", report)

Classification report without oversampling: 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     56857
           1       0.81      0.72      0.76       105

    accuracy                           1.00     56962
   macro avg       0.90      0.86      0.88     56962
weighted avg       1.00      1.00      1.00     56962



**Precision** refers to how what percentage of positive class identifications are actually correct while **recall** refers to what percentage of positive classes are identified correctly. And F1-score combines the precision and recall score together.


According to this, the precision and recall is excellent for the non-fraudulent cases and somewhat okay for the fraudulent cases. The F1-score seem to suggest that overall, the model did decent.

Let's do one last check to see how this model of no oversampling fares by the ROC curve.

ROC curve is a performance measurement which tells how capable the model is at distinguishing different classes. It scores between 0.5 to 1.0

In [ ]:
auc_score = roc_auc_score(test_labels, predictions)

print("AUC score without oversampling:", auc_score)

AUC score without oversampling: 0.8617464700497574


This area-under-the-curve (AUC) score is rather good. This suggests that the model here can be used for credit card fraud detection, but that does not mean we cannot do better. 

#Applying Scikit-learn's Logistic Regression **with** SMOTE sampling

As mentioned earlier, this problem has a huge class imbalance and the model may not be trained properly due to this imbalance. Therefore, it is now time to implement SMOTE oversampling, and hopefully, the algorithm would improve.

We need to now create new variables for the **training** set using SMOTE.



In [ ]:
smote = SMOTE(random_state = 10, sampling_strategy = 1.0)
training_features_smote, training_labels_smote = smote.fit_resample(training_features, training_labels)

Now we just need to repeat what we did earlier, except with new SMOTE training set variables.

Creating a logistic regression model with SMOTE variables.

In [ ]:
logreg_smote = LogisticRegression(max_iter=1000)
logreg_smote.fit(training_features_smote, training_labels_smote)

logreg_score_smote = logreg_smote.score(test_features, test_labels)
print("Logistic regression accuracy with oversampling:", logreg_score_smote)

Logistic regression accuracy with oversampling: 0.9825146588954039


We see here that the logistic regression model with SMOTE resulted in a slightly lower overall accuracy than the model without SMOTE.

This might be due to the model sacrificing some accuracy to better predict the class that we are interested in due to it being oversampled.

In [ ]:
cv_scores_smote = cross_val_score(logreg_smote, training_features_smote, training_labels_smote, cv=10)

print("Cross Validation Scores with oversampling: \n", cv_scores_smote)

cv_scores_smote = pd.Series(cv_scores_smote)
print("Cross Validation Minimum:", cv_scores_smote.min())
print("Cross Validation Mean:", cv_scores_smote.mean()) 
print("Cross Validation Max:", cv_scores_smote.max())

Cross Validation Scores with oversampling: 
 [0.97247868 0.95414578 0.97410534 0.9695331  0.96975292 0.97340192
 0.95284782 0.97164274 0.9734453  0.95495812]
Cross Validation Minimum: 0.9528478160515267
Cross Validation Mean: 0.9666311722807948
Cross Validation Max: 0.9741053372021454


Like before, the cross validation results are producing similar accuracy as to our trained SMOTE model's accuracy. 

Now let's take a look if the model is actually accurately predicting the fraudulent cases correctly. 

In [ ]:
predictions_smote = logreg_smote.predict(test_features)

cm_smote =  confusion_matrix(test_labels, predictions_smote)

print("Confusion matrix with oversampling: \n", cm_smote)

Confusion matrix with oversampling: 
 [[55869   988]
 [    8    97]]


As a reminder...

**Upper Left Corner**: The amount of correctly classified by the model for non-fraudulent transactions.

**Upper Right Corner**: The amount of incorrectly classified as fraudulent transactions, but the actual label is non-fraudulent.

**Lower Left Corner**: The amount of incorrectly classified as non-fraudulent transactions, but the actual label is fraudulent.

**Lower Right Corner**: The amount of correctly classified by the model for fraudulent transactions.

As a result of the model being trained through SMOTE, the model has at least a 90% accuracy in predicting in the class that we are interested in! That is a big improvement compared to previous 70% accuracy.

However, we do see that it is predicting more non-fraudulent cases as fraudulent. But, that is fine since it is better to be safe than sorry.

Credit card companies can better protect their consumers and get people to verify their transactions if the algorithm think it is fraudulent case.

Now let's see what the classification report says.

In [ ]:
report_smote = classification_report(test_labels, predictions_smote)

print("Classification report with oversampling: \n", report_smote)

Classification report without oversampling: 
               precision    recall  f1-score   support

           0       1.00      0.98      0.99     56857
           1       0.09      0.92      0.16       105

    accuracy                           0.98     56962
   macro avg       0.54      0.95      0.58     56962
weighted avg       1.00      0.98      0.99     56962



As another a reminder...

**Precision** refers to how what percentage of positive class identifications are actually correct while **recall** refers to what percentage of positive classes are identified correctly. And F1-score combines the precision and recall score together.

We see here that the results further reinforce what I was saying earlier. The low score in precision resulted from the model predicting more non-fraudulent cases as fraudulent. But in exchange, the model is predicting fraudulent cases much more accurately. The F-score may look very poor, but in actuality, this is a much better model than the one without oversampling.

Lastly, let's see what the ROC curve suggests about the model.

In [ ]:
auc_score_smote = roc_auc_score(test_labels, predictions_smote)

print("AUC score with oversampling:", auc_score_smote)

AUC score without oversampling: 0.953216297863395


This AUC score is quite excellent and would be great one to use in the industry.

When compared to the previous score, this score suggests that this model, which was trained through oversampling the minority class, is the better one.

#Conclusions

In conclusion, logistic regression is a great model to use for credit card detection.

Since there was a huge class imbalance, training the model with SMOTE oversampling yielded a much better algorithm for the class that we are interested in than training the model without it.

Now let's see which features in particular heavily influenced if a transaction was fraudulent or not.

In [ ]:
#this contains all the weights in algorithm
weights = logreg_smote.coef_

#this contains all the feature names
features = df.columns.values

#creates a dictionary of the features and their associated weights
weights_dict = {}
for weight, feat in zip(weights[0,:], features):
    weights_dict[feat] = weight

#prints all key value pairs in dictionary
for element in weights_dict.items():
  print(element)

('Time', -3.757874976432943e-05)
('V1', 0.15019753375998096)
('V2', -0.2489840494760921)
('V3', -0.8587742651232082)
('V4', 0.8637960973558975)
('V5', 0.26399804892451945)
('V6', -0.3029587023427311)
('V7', -0.23103062793153795)
('V8', -0.4395015697357681)
('V9', -0.5360104612444051)
('V10', -0.6424805605493271)
('V11', 0.2349376690388481)
('V12', -0.8696767603767634)
('V13', -0.44404124544872386)
('V14', -1.4416911904021594)
('V15', -0.2613064095415009)
('V16', -0.6275713551943845)
('V17', -1.0287285616573794)
('V18', -0.12192251237622763)
('V19', 0.2052061882717862)
('V20', 0.03372111805303771)
('V21', 0.11090756685767993)
('V22', 0.36430690113643127)
('V23', 0.06889949398808184)
('V24', 0.08887837331977193)
('V25', -0.2228668997492212)
('V26', -0.06125643451590009)
('V27', 0.07206251068071699)
('V28', 0.0333622415501257)
('Amount', -0.0004798187593282112)


Since this is logistic regression, negative weights would make the model incline to output a 0 (aka the class we are NOT interested in) while positive weights would make the model incline to output a 1 (aka the class we are interested in).

The features "Time", "V14", and "V17" seem to be the most significant features to influencing non-fraudulent cases.

The features "V4", "V19", and "V22" seem to be the most significant features to influencing fraudulent cases.